In [ ]:
def add(x,y):
        print("add func")
        return x+y
decorators.add = logged(decorators.add)
    ## Same as
    

@logged
def add(x,y):
        print("add func")
        return x+y

    

In [ ]:
## Decorators

def null_decorator(func):
    return func

def greet():
    return 'Hello!'

greet = null_decorator(greet)

In [ ]:
greet()

In [ ]:
## Above same as this ..

@null_decorator
def greet():
    return 'Hello!'


In [ ]:
greet()

In [ ]:
## Decorators


In [ ]:
from functools import wraps
def nullable(function):
    @wraps(function)
    def null_wrapper(arg):
        return None if arg is None else function(arg)
    return null_wrapper

In [ ]:
import math
nlog = nullable(math.log)

In [ ]:
some_data = [10, 100, None, 50, 60]

In [ ]:
scaled = map(nlog, some_data) 

In [ ]:
list(scaled)

In [ ]:
## Enter Context Manager

In [2]:
## Context Manager

f = open("x.txt")

In [3]:
f.__enter__()

<_io.TextIOWrapper name='x.txt' mode='r' encoding='UTF-8'>

In [4]:
f.read(1)

''

In [5]:
## now lets run exit to the close the file
f.__exit__(None, None, None)

In [6]:
## try to read once again
f.read(1)

ValueError: I/O operation on closed file.

In [7]:
## Context Manager Summary 
## so to open a file, process its contents, and make sure to close it, 
## you can simply do:

with open("x.txt") as f:
    data = f.read()
    do something with data

SyntaxError: invalid syntax (<ipython-input-7-c16462b4ded2>, line 7)

In [8]:
## Acquiring a lock during threading

import threading
lock=threading.Lock()
lock.acquire()

True

In [9]:
print("Use the lock")
lock.release()

## you acquire a resource, use it and then close it. 
## but it is easy to close or release the resources
## Context Manager


Use the lock


In [11]:
## Context Manager

class Context(object):

    def __init__(self, handle_error):
        print ('__init__(%s)' % handle_error)
        self.handle_error = handle_error
    def __enter__(self):
        print ('__enter__()')
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        print ('__exit__(%s, %s, %s)' % (exc_type, exc_val, exc_tb))
        return self.handle_error

In [14]:
with Context(True) as foo:
    print ('This is in the context')
    ## run once and the uncomment following line
    raise RuntimeError('this is the error message')

__init__(True)
__enter__()
This is in the context
__exit__(<class 'RuntimeError'>, this is the error message, <traceback object at 0x106b42cc8>)


In [15]:
## In the above example your code is bracketed by enter and exit

## Because the exit method returns True, the raised error is ‘handled’.
## What if we try with False?

with Context(False) as foo:
    print ('This is in the context')
    ## run this with and without commenting following line
    raise RuntimeError('this is the error message')

__init__(False)
__enter__()
This is in the context
__exit__(<class 'RuntimeError'>, this is the error message, <traceback object at 0x106b42b48>)


RuntimeError: this is the error message

In [ ]:
## contextlib.contextmanager() 
## will turn a generator function into context manager.

In [16]:
from contextlib import contextmanager

@contextmanager
def context(boolean):
    print ("__init__ code here")
    try:
        print ("__enter__ code goes here")
        yield object()
    except Exception as e:
        print ("errors handled here")
        if not boolean:
            raise
    finally:
        print ("__exit__ cleanup goes here")


In [17]:
import contextlib
import http.client
with contextlib.closing(
  http.client.HTTPConnection("www.example.com")) as host:
    host.request("GET", "/path/to/resources/12345/")
    response= host.getresponse()
    print(response.read())

b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 50px;\n        background-color: #fff;\n        border-radius: 1em;\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        body {\n            background-color: #fff;\n        }\n        div {\n            width: auto;\n            margin: 0 auto;\n            border-radius: 0;\n            padding: 1em;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\

In [21]:
from __future__ import print_function
import contextlib

@contextlib.contextmanager
def manager():
    """Easiest way to get a custom context manager..."""
    try:
        print('Entered')
        yield
    finally:
        print('Closed')


def gen():
    """Just a generator with a context manager inside.

    When the context is entered, we'll see "Entered" on the console
    and when exited, we'll see "Closed" on the console.
    """
    man = manager()
    with man:
        for i in range(10):
            yield i


# Test what happens when we consume a generator.
list(gen())

def fn():
    g = gen()
    next(g)
    # g.close()

# Test what happens when the generator gets garbage collected inside
# a function
print('Start of Function')
fn()
print('End of Function')

# Test what happens when a generator gets garbage collected outside
# a function.  IIRC, this isn't _guaranteed_ to happen in all cases.
g = gen()
next(g)
# g.close()
print('EOF')

Entered
Closed
Start of Function
Entered
Closed
End of Function
Entered
EOF


In [20]:
## Context Manager Examples - Advanced Usecase. 
## yield without argument is semantically equivalent to yield None

from contextlib import contextmanager
import sys

@contextmanager
def redirected(**kwds):
    stream_names = ["stdin", "stdout", "stderr"]
    old_streams = {}
    try:
        for sname in stream_names:
            stream = kwds.get(sname, None)
            if stream is not None and stream != getattr(sys, sname):
                old_streams[sname] = getattr(sys, sname)
                setattr(sys, sname, stream)
        yield
    finally:
        for sname, stream in old_streams.items():
            setattr(sys, sname, stream)

with redirected(stdout=open("./uw-py220-log-context-mgr.txt", "w")):
     # these print statements will go to /tmp/log.txt
     print ("Test entry 1")
     print ("Test entry 2")
# back to the normal stdout
print ("Back to normal stdout again")

Back to normal stdout again


In [ ]:
## Yield in Context Manager

yield expression returns control to the whatever is using the generator. 
The generator pauses at this point, which means that the @contextmanager 
decorator knows that the code is done with the setup part.

In other words, everything you want to do in the context manager __enter__ 
phase has to take place before the yield.

Once your context exits (so the block under the with statement is done), 
the @contextmanager decorator is called for the __exit__ part of the 
context manager protocol and will do one of two things:

If there was no exception, it'll resume your generator. So your generator 
unpauses at the yield line, and you enter the cleanup phase, the part

If there was an exception, the decorator uses generator.throw() to raise 
that exception in the generator. It'll be as if the yield line caused 
that exception. Because you have a finally clause, it'll be executed 
before your generator exits because of the exception.

In [ ]:
## Recursion

## Recursion to use up all stack space. 
## RuntimeError: maximum recursion depth exceeded


In [ ]:
def f():
    print("Hello")
    f()

f()